# Open source quantum development with Codespaces

This notebook walks through a simple demonstration of how to go from user developed quantum programs to submitting them to be run on cloud resources, all from Python and Jupyter Notebooks.


---

## Develop quantum programs in Python

In [1]:
from qutip_qip.circuit import QubitCircuit
from qutip_qip.qir import circuit_to_qir

https://qutip-qip.readthedocs.io/en/latest/qip-simulator.html?highlight=qir#import-and-export-quantum-circuits

In [6]:
#Playing the CHSH game 
circuit = QubitCircuit(3, num_cbits=2)
msg, here, there = range(3)

# Encode message
circuit.add_gate("RZ", targets=[msg], arg_value=0.123)

# Entangle qubits
circuit.add_gate("SNOT", targets=[here])
circuit.add_gate("CNOT", targets=[there], controls=[here])
circuit.add_gate("CNOT", targets=[here], controls=[msg])
circuit.add_gate("SNOT", targets=[msg])

# Players measure their qubits
circuit.add_measurement("Z", targets=[msg], classical_store=0)
circuit.add_measurement("Z", targets=[here], classical_store=1)

with open('output/qutip_program_simple.bc', 'wb') as file:
    file.write(circuit_to_qir(circuit, "bitcode"))
    
with open('output/qutip_program_simple.ll', 'w') as file:
    file.write(circuit_to_qir(circuit, "text"))


---

## Compile programs to QIR

---

## Optimize programs

Great! we now have a way of lowering high-level user code to QIR, but what does that look like?
Let's check out part of the QIR file we just generated:

In [7]:
with open('output/qutip_program_simple.ll', 'r') as f:
    print(f.read())

; ModuleID = 'qutip_circuit'
source_filename = "qutip_circuit"

%Result = type opaque
%Qubit = type opaque

declare void @__quantum__rt__array_start_record_output()

declare void @__quantum__rt__array_end_record_output()

declare void @__quantum__rt__result_record_output(%Result*)

define void @main() #0 {
entry:
  call void @__quantum__qis__rz__body(double 1.230000e-01, %Qubit* null)
  call void @__quantum__qis__h__body(%Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__cnot__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Qubit* inttoptr (i64 2 to %Qubit*))
  call void @__quantum__qis__cnot__body(%Qubit* null, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__h__body(%Qubit* null)
  call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  call void @__quantum__qis__mz__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Result* inttoptr (i64 1 to %Result*))
  call void @__quantum__rt__array_start_record_output()
  call void @__quantum__rt__result_record_out

This isn't the nicest way to debug things (though really it should be only the compiler looking). Let's look at a control flow graph:

In [4]:
!opt -dot-cfg output/qutip_program.ll 

This is inadvisable as it may cause display problems. If
you REALLY want to taste LLVM bitcode first-hand, you
can force output with the `-f' option.

Writing '.main.dot'...


In [5]:
!dot -Tpng output/.main.dot -ooutput/qutip_program.png

Error: dot: can't open output/.main.dot: No such file or directory


![flow-diagram](qutip_program.png)

In [6]:
! qat --apply -S ./qutip_program.ll -o ./qutip_program_optimized.ll --save-config

Failed to load /workspaces/universe-qir-demo/./qutip_program.ll
Could not load ./qutip_program.ll


In [7]:
import os

from numpy import size
print(f'Before: {os.stat("qutip_program.ll").st_size}, {os.stat("qutip_program_optimized.ll").st_size}')

FileNotFoundError: [Errno 2] No such file or directory: 'qutip_program.ll'

In [ ]:
! qat --apply --profile base -S ./qutip_program.ll

---

## Submitting quantum programs to the cloud

Now that we have generated our qir file, we want to submit it to a cloud provider to run it against a simulator or hardware device.

We start by importing the appropriate Python package to work connect to an Azure Quantum resource.

In [8]:
with open('./output/qutip_program_simple.bc', 'rb') as f:
    bitcode = f.read()

In [9]:
from azure.identity import AzureCliCredential

NOTE: You will have to open a terminal and run `az login` to make sure your Azure credientials are stored.

In [10]:
import azure.quantum
workspace = azure.quantum.Workspace(
    subscription_id="bd1621e9-9094-48e4-99f4-00256be51682",
    resource_group="AzureQuantum",
    name="qir-demo",
    location="eastus",
    credential=AzureCliCredential()
)

In [12]:
job = azure.quantum.Job.from_input_data(
    workspace=workspace,
    name="qir-demo",
    provider_id="rigetti",
    target="rigetti.sim.qvm",
    input_data_format="qir.v1",
    output_data_format="microsoft.quantum-results.v1",
    input_data=bitcode,
    input_params={
        "shots": 500,
        "entryPoint": "main",
        "arguments": [],
        # "targetCapability": "AdaptiveExecution"
    }
)

In [13]:
job.id

'55c53846-4d7a-11ed-a569-0242ac110002'

In [14]:
job.get_results()

........

{'Histogram': ['[1, 0]', 1.0]}